## Import the necessary packages

Set the matplotlib backend so figures can be saved in the background

In [1]:
import matplotlib
matplotlib.use("Agg")

In [2]:
import sys
sys.path.append("..")
import os
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from nn.conv.minivggnet import MiniVGGNet
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

Define learning rate scheduler

In [3]:
def step_decay(epoch):
    # initialize the base initial learning rate, drop factor, and
    # epochs to drop every
    initAlpha = 0.01
    factor = 0.25
    dropEvery = 5

    # compute learning rate for the current epoch
    alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))

    # return the learning rate
    return float(alpha)

Construct the dictionary instead of argument parse

In [4]:
args = {
    "output" : "output/lr_decay_f0.25_plot.png"
}

Load the training and testing data, then scale it into the range [0, 1]

In [5]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...


Convert the labels from integers to vectors

In [6]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

Initialize the label names for the CIFAR-10 dataset

In [7]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

## Let's compile our model and start training MiniVGGNet with Learning rate decay:

Define the set of callbacks to be passed to the model during training

In [8]:
callbacks = [LearningRateScheduler(step_decay)]

Initialize the optimizer and model

In [9]:
print("[INFO] compiling model...")
opt = SGD(lr=0.01, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


d:\tu hoc\deep learning for computer vision with python\starter_bundle\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Train the network

In [10]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=40, callbacks = callbacks, verbose=1)

[INFO] training network...
Epoch 1/40
782/782 [==============================] - 26s 24ms/step - loss: 1.6099 - accuracy: 0.4664 - val_loss: 1.2633 - val_accuracy: 0.5755
Epoch 2/40
782/782 [==============================] - 17s 22ms/step - loss: 1.1318 - accuracy: 0.6138 - val_loss: 1.1340 - val_accuracy: 0.6074
Epoch 3/40
782/782 [==============================] - 17s 22ms/step - loss: 0.9621 - accuracy: 0.6676 - val_loss: 0.8717 - val_accuracy: 0.6925
Epoch 4/40
782/782 [==============================] - 17s 22ms/step - loss: 0.8592 - accuracy: 0.7029 - val_loss: 0.7464 - val_accuracy: 0.7431
Epoch 5/40
782/782 [==============================] - 17s 22ms/step - loss: 0.7102 - accuracy: 0.7506 - val_loss: 0.6613 - val_accuracy: 0.7701
Epoch 6/40
782/782 [==============================] - 18s 23ms/step - loss: 0.6617 - accuracy: 0.7676 - val_loss: 0.6307 - val_accuracy: 0.7816
Epoch 7/40
782/782 [==============================] - 17s 22ms/step - loss: 0.6342 - accuracy: 0.7760 - val_l

Evaluate the network

In [11]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.82      0.80      0.81      1000
  automobile       0.92      0.88      0.90      1000
        bird       0.74      0.67      0.70      1000
         cat       0.65      0.60      0.63      1000
        deer       0.74      0.82      0.78      1000
         dog       0.71      0.73      0.72      1000
        frog       0.80      0.89      0.85      1000
       horse       0.86      0.83      0.85      1000
        ship       0.89      0.90      0.89      1000
       truck       0.85      0.89      0.87      1000

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



Plot the training loss and accuracy

In [13]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 40), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 40), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10 with Learning rate decay")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])